In [213]:
import re
import requests
from bs4 import BeautifulSoup
import pandas as pd
from konlpy.tag import Okt  
okt = Okt() 
import tensorflow as tf
import numpy as np
from collections import Counter

from wordcloud import WordCloud
import matplotlib.pyplot as plt


import urllib.request
from tqdm import tqdm
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer

AttributeError: module 'tweepy' has no attribute 'StreamListener'

In [427]:
import matplotlib.pyplot as plt 
%matplotlib inline
from string import punctuation
import warnings
warnings.filterwarnings('ignore')

In [41]:
total_playlist = pd.read_csv('total_playlist.csv')

In [42]:
total_playlist.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27927 entries, 0 to 27926
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   song_id     27927 non-null  int64 
 1   song_name   27927 non-null  object
 2   artist      27927 non-null  object
 3   album       27926 non-null  object
 4   Like_Count  27918 non-null  object
 5   Lyric       27894 non-null  object
 6   cover_url   27906 non-null  object
 7   tags        27906 non-null  object
dtypes: int64(1), object(7)
memory usage: 1.7+ MB


In [473]:
 # 불용어 (가사 빈도수 높은 + 감정분류와 무관한 단어 추가 중)
stop_w = ['all','이렇게','네가','있는','니가','없는','너의','너무','그런',
          'oh','whoo','tuesday','내가','너를','나를','we','this','the','그렇게',
          'so','am','baby','and','can','you','much','me','for','go','in',
          '은', '는', '이', '가', '하','부터','처럼','까지',
          'know','no','of','let','my','수','너','내','나','그','난','봐',
          '돼','건','모든','에서','에게','싶어','잖아',
          '날','널','수','것','못','말','넌','젠','하나','정말','알','여기',
          '다시','하게','니까',
          '때','아','더','게','또','채','일','걸','누구','나는','너는','라면',
          '같아','있어',
          '의','가','보','들','좀','잘','걍','과','도','를','으로','우린','하지',
          '해도','하고','없어','않아',
          '자','에','와','한','하다','네','있다','나의','해','다','내게','왜',
          '거야','이제','그냥','했던','하는']
# 우리 , 그대 , 사랑 , 지금

In [44]:
total_playlist['Lyric'] = total_playlist['Lyric'].apply(lambda x: " ".join(x.lower() for x in str(x).split() if x not in stop_w))



In [45]:
words = []
for word in total_playlist['Lyric'].tolist():
    words.append(word.split())
    words = [each_word for each_word in words if each_word not in stop_w]
#create the new column with the information of words lists
total_playlist['words'] = words
total_playlist.head()

,song_id,song_name,artist,album,Like_Count,Lyric,cover_url,tags,words
0,30392514,bath,offonoff (오프온오프),bath,"[5,781]",거긴 날씨가 어때 하늘과 밤엔 자꾸 떠오르는 기억이 계속 말을 건네네 우리는 함께일...,https://image.bugsm.co.kr/album/images/200/200...,"['국내', '힙합', '알앤비', '밤/새벽', '감각적인', '작업실', '몽환...","[거긴, 날씨가, 어때, 하늘과, 밤엔, 자꾸, 떠오르는, 기억이, 계속, 말을, ..."
1,30346372,Belief (millic remix),millic(밀릭),Belief,"[1,510]",our belief each other written on a page was to...,https://image.bugsm.co.kr/album/images/200/200...,"['국내', '힙합', '알앤비', '밤/새벽', '감각적인', '작업실', '몽환...","[our, belief, each, other, written, on, a, pag..."
2,30061516,[권리없는 곡],아파치(Apachi),Toujours Youth Remix Album,[54],겁이 아무 말이나 들려줘 girl i'm tired lgbt next to 순전히 ...,https://image.bugsm.co.kr/album/images/200/200...,"['국내', '힙합', '알앤비', '밤/새벽', '감각적인', '작업실', '몽환...","[겁이, 아무, 말이나, 들려줘, girl, i'm, tired, lgbt, nex..."
3,30506160,Every,민제 (Minje),Boy II Man,[317],i don't what? every time everything do everyth...,https://image.bugsm.co.kr/album/images/200/200...,"['국내', '힙합', '알앤비', '밤/새벽', '감각적인', '작업실', '몽환...","[i, don't, what?, every, time, everything, do,..."
4,3988024,On & On (Prod. By The Quiett),육지담,언프리티 랩스타 Semi Final 2,[581],첨부터 길을 잃었던 아이 세상과 마주하기에 어렸던 나이 너넨 모르겠지 tv에 웃는 ...,https://image.bugsm.co.kr/album/images/200/493...,"['국내', '힙합', '알앤비', '밤/새벽', '감각적인', '작업실', '몽환...","[첨부터, 길을, 잃었던, 아이, 세상과, 마주하기에, 어렸던, 나이, 너넨, 모르..."


In [46]:
total_playlist.to_csv('total_playlist_word.csv',index=False)

In [47]:
print(total_playlist.isnull().sum())

song_id        0
song_name      0
artist         0
album          1
Like_Count     9
Lyric          0
cover_url     21
tags          21
words          0
dtype: int64


In [48]:
total_playlist.loc[total_playlist.Lyric.isnull()] # 가사없음

,song_id,song_name,artist,album,Like_Count,Lyric,cover_url,tags,words


In [49]:
total_playlist['Lyric'] = total_playlist['Lyric'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")
# 한글과 공백을 제외하고 모두 제거
total_playlist[:5]

,song_id,song_name,artist,album,Like_Count,Lyric,cover_url,tags,words
0,30392514,bath,offonoff (오프온오프),bath,"[5,781]",거긴 날씨가 어때 하늘과 밤엔 자꾸 떠오르는 기억이 계속 말을 건네네 우리는 함께일...,https://image.bugsm.co.kr/album/images/200/200...,"['국내', '힙합', '알앤비', '밤/새벽', '감각적인', '작업실', '몽환...","[거긴, 날씨가, 어때, 하늘과, 밤엔, 자꾸, 떠오르는, 기억이, 계속, 말을, ..."
1,30346372,Belief (millic remix),millic(밀릭),Belief,"[1,510]",...,https://image.bugsm.co.kr/album/images/200/200...,"['국내', '힙합', '알앤비', '밤/새벽', '감각적인', '작업실', '몽환...","[our, belief, each, other, written, on, a, pag..."
2,30061516,[권리없는 곡],아파치(Apachi),Toujours Youth Remix Album,[54],겁이 아무 말이나 들려줘 순전히 아름다움 팍 튀고 팍 어쩔 없이 감...,https://image.bugsm.co.kr/album/images/200/200...,"['국내', '힙합', '알앤비', '밤/새벽', '감각적인', '작업실', '몽환...","[겁이, 아무, 말이나, 들려줘, girl, i'm, tired, lgbt, nex..."
3,30506160,Every,민제 (Minje),Boy II Man,[317],길이 어두운 어두운 어두운 ...,https://image.bugsm.co.kr/album/images/200/200...,"['국내', '힙합', '알앤비', '밤/새벽', '감각적인', '작업실', '몽환...","[i, don't, what?, every, time, everything, do,..."
4,3988024,On & On (Prod. By The Quiett),육지담,언프리티 랩스타 Semi Final 2,[581],첨부터 길을 잃었던 아이 세상과 마주하기에 어렸던 나이 너넨 모르겠지 에 웃는 모습...,https://image.bugsm.co.kr/album/images/200/493...,"['국내', '힙합', '알앤비', '밤/새벽', '감각적인', '작업실', '몽환...","[첨부터, 길을, 잃었던, 아이, 세상과, 마주하기에, 어렸던, 나이, 너넨, 모르..."


In [51]:
import numpy as np

In [52]:
total_playlist['Lyric'] = total_playlist['Lyric'].str.replace('^ +', "") # white space 데이터를 empty value로 변경
total_playlist['Lyric'].replace('', np.nan, inplace=True)
print(total_playlist.isnull().sum())

song_id          0
song_name        0
artist           0
album            1
Like_Count       9
Lyric         1124
cover_url       21
tags            21
words            0
dtype: int64


In [56]:
total_playlist = total_playlist[~total_playlist['Lyric'].isnull()] # 가사가 영어로만 이루어져 있어서 NaN값인 노래들

In [57]:
total_playlist = total_playlist.dropna(how = 'any')

In [92]:
total_playlist.drop_duplicates(['song_id','Lyric'], keep='first',inplace=True)

In [93]:
print('전처리 후 데이터의 개수 :',len(total_playlist))

전처리 후 데이터의 개수 : 15877


In [209]:
total_playlist.drop_duplicates(['song_id'], keep='first',inplace=True)

In [210]:
print('전처리 후 데이터의 개수 :',len(total_playlist))

전처리 후 데이터의 개수 : 15341


In [94]:
total_playlist = total_playlist.reset_index(drop=True)

In [59]:
# 불용어 (가사 빈도수 높은 + 감정분류와 무관한 단어 추가 중)
stop_w = ['all','이렇게','네가','있는','니가','없는','너의','너무','그런',
          'oh','whoo','tuesday','내가','너를','나를','we','this','the','그렇게',
          'so','am','baby','and','can','you','much','me','for','go','in',
          '은', '는', '이', '가', '하',
          'know','no','of','let','my','사랑','수','너','내','나','그','난','봐','돼','건','모든',
          '날','널','수','것','못','말','넌','젠','하나','정말','알','여기','우리','그대','다시',
          '때','아','더','게','또','채','일','걸','누구','나는','너는','라면','같아','있어','지금',
          '의','가','보','들','좀','잘','걍','과','도','를','으로','우린',
          '자','에','와','한','하다','네','있다','나의','해','다','내게','왜','거야']

In [60]:
# 불용어 단어 중복제거
stop_w = set(stop_w)

In [436]:
"""TF-IDF로 만들기"""
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer()
tfidf_matrix = tfidf.fit_transform(total_playlist.Lyric)
# 줄거리에 대해서 tf-idf 수행
print(tfidf_matrix.shape)

(15341, 139863)


In [437]:
"""코사인 유사도 구하기"""
from sklearn.metrics.pairwise import linear_kernel
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [438]:
"""인덱스 테이블 만들기"""
indices = pd.Series(total_playlist.index, index=total_playlist.song_name).drop_duplicates()
print(indices)

song_name
bath                                 0
[권리없는 곡]                             1
Every                                2
On & On (Prod. By The Quiett)        3
NOT TODAY                            4
                                 ...  
떠날랏꼬                             15336
괜찮아 떠나                           15337
배낭여행                             15338
좌절금지                             15339
아프리카 청춘이다                        15340
Length: 15341, dtype: int64


In [439]:
indices[453:463]

song_name
Why                              453
Free Somebody                    454
Shut Up & Groove (feat. DEAN)    455
한숨                               456
I Just Wanna Dance               457
쏘쏘                               458
농담 반 진담 반                        459
봄이 좋냐??                          460
어떻게 생각해                          461
니 눈 니 코 니 입술 (feat. 79)          462
dtype: int64

In [98]:
total_playlist.head(1)

,song_id,song_name,artist,album,Like_Count,Lyric,cover_url,tags,words
0,30392514,bath,offonoff (오프온오프),bath,"[5,781]",거긴 날씨가 어때 하늘과 밤엔 자꾸 떠오르는 기억이 계속 말을 건네네 우리는 함께일...,https://image.bugsm.co.kr/album/images/200/200...,"['국내', '힙합', '알앤비', '밤/새벽', '감각적인', '작업실', '몽환...","[거긴, 날씨가, 어때, 하늘과, 밤엔, 자꾸, 떠오르는, 기억이, 계속, 말을, ..."


In [442]:
"""추천 해주기"""
def music_REC(song_name, cosine_sim=cosine_sim):
    #입력한 노래로부터 인덱스 가져오기
    idx = indices[song_name]

    # 모든 노래에 대해서 해당 노래와의 유사도를 구하기
    sim_scores = list(enumerate(cosine_sim[idx]))

    # 유사도에 따라 노래들을 정렬
    sim_scores = sorted(sim_scores, key=lambda x:x[1], reverse = True)

    # 가장 유사한 10개의 노래를 받아옴
    sim_scores = sim_scores[1:11]

    # 가장 유사한 10개 노래의 인덱스 받아옴
    music_indices = [i[0] for i in sim_scores]
    
    # 기존에 읽어들인 데이터에서 해당 인덱스의 값들을 가져온다. 
    # 그리고 스코어 열을 추가하여 코사인 유사도도 확인할 수 있게 한다.
    result_df = total_playlist.iloc[music_indices].copy()
    result_df['score'] = [i[1] for i in sim_scores]
    
    # 읽어들인 데이터에서 제거할 칼럼명
    # del result_df['content']
    del result_df['cover_url']
    del result_df['album'] 
    del result_df['words'] 

    # 가장 유사한 10개의 노래의 제목을 리턴
    return result_df

In [443]:
music_REC("I Just Wanna Dance")

,song_id,song_name,artist,Like_Count,Lyric,tags,score
4944,53081,점점 더,장필순,[228],저녁 창가에 부서지는 햇살은 눈부시지 눈을 감을수록 점점 아침 창밖에 지저귀는 새들...,"['국내', '복고/레트로', '뉴트로', '시티팝', '신스팝', '가요', '인...",0.119407
11308,3673403,아드레날린 (Adrenaline),소녀시대-태티서(Girls' Generation-TTS),"[2,032]",이런 느낌은 뭐야 나만 모르는 감춰둔 블랙홀 새로운 세계가 반겨 이건 심장의 장난 ...,"['국내', '가요', '신나는', '아이돌', '운동/헬스', '드라이브', '노...",0.116888
6759,85257,바꿔,이정현,"[21,920]",모두 제정신이 아니야 다들 미쳐가고만 어느 누굴 믿어 어찌믿어 더는 못믿어 누가 누...,"['국내', '가요', '댄스', '테크노', '드라이브', '스트레스/짜증', '...",0.115246
8411,30892394,In The Moonlight,식스밤(Six Bomb),[24],너에게 취해있어 너에게 빠져있어 자꾸 눈이 자꾸 맘이 ...,"['국내', '시티팝', '가요', '선선한', '여름', '여름밤', '밤/새벽'...",0.114511
6933,4586784,리듬 타 (RHYTHM TA),iKON,"[3,700]",이건 그냥 노래니깐 리듬 타 리듬 타 리듬 타 리듬 타 이건 그냥 노래니깐 리...,"['국내', '가요', 'KPOP', '아이돌', '강한', '스트레스/짜증', '...",0.097411
11440,30107396,리듬 타 (RHYTHM TA) REMIX (Rock Ver.),iKON,[688],이건 그냥 노래니깐 리듬 타 리듬 타 리듬 타 리듬 타 이건 그냥 노래니깐 리...,"['국내', '가요', '아이돌', 'KPOP', '신나는', '노동요', '운동/...",0.097411
13880,2967611,하.니.뿐.,2PM,[942],하 하 하루 종일 니 니 생각 뿐이야 뿐이야 뿐이야 뿐이야 하 하 하루 종일 니 니...,"['가요', '그리움']",0.097088
7088,1897561,바꿔,글루미 써티스(Gloomy 30's),"[8,970]",태양은 회색빛 구름에 가려 빛을 잃고 세상은 깊은 시름 속에서 춤을 추네 바꿔 이젠...,"['국내', '락', '신나는', '스트레스/짜증', '폭염/더위', '시원한', ...",0.096990
116,31642250,움직여 (MOVE) (Prod. by ZICO),SIXC (6 crazy),"[1,852]",시작하는 선과 끝점이 같아야 원이 그려져 둥근 모양이 차츰 정교해질 곧 지구라...,"['국내', '가요', 'KPOP', '아이돌', '치명적인', '섹시한', '남자...",0.094095
5401,30788186,Girl Front,이달의 소녀 오드아이써클,[761],눈을 뗄 수가 없어 너와 내 눈에만 보이던 너 손댈 없이 위험 했던 맘도 너에...,"['국내', '가요', 'KPOP', '아이돌', '청량한', '시원한', '여름'...",0.093276


In [133]:
music_REC("리듬 타 (RHYTHM TA)")

,song_id,song_name,artist,Like_Count,Lyric,tags,score
11944,30107396,리듬 타 (RHYTHM TA) REMIX (Rock Ver.),iKON,[688],이건 그냥 노래니깐 리듬 타 리듬 타 리듬 타 리듬 타 이건 그냥 노래니깐 리...,"['국내', '가요', '아이돌', 'KPOP', '신나는', '노동요', '운동/...",1.000000
2080,31369191,"Hair in the Air (Sung By 예리, 런쥔, 제노, 재민)",예리 (YERI),[890],어서 나와 빨리 나와 뛰어놀자 하늘 닿도록 너와 나와 모두 나와 같이 놀자 볼륨 키...,"['국내', '아이돌', '가요', 'KPOP', '기분전환', '드라이브', '월...",0.177090
4920,31639081,Midnight,에일리(Ailee),[211],두 눈처럼 새까만 밤은 어느새 우릴 감싸오고 쏟아지는 별 셀 없이 바랬던 순간 듣고...,"['국내', '아이돌', '댄스', '일렉트로닉팝', '신나는', '드라이브', '...",0.160874
8138,3986111,Kangaroo,라이너스의 담요(Linus' Blanket),[145],처음으로 돌아가 이제부터 시작해요 설명은 필요도 없죠 너와 함께라면 ...,"['국내', '인디', '국내 인디', '상쾌한', '청량한', '화창한 날', '...",0.158346
5992,5030288,Spin Up!,SUPER JUNIOR (슈퍼주니어),[349],오늘도 쌓여 있다면 여기로 모여 성별 직업 필요 없어 어차피 밤은 시작됐으니까...,"['국내', '레이블', 'SM', '아이돌', '가요', '여름', '청량한', ...",0.149409


In [176]:
music_REC("Joa Yo!").head(5)

,song_id,song_name,artist,Like_Count,Lyric,tags,score
13986,5951260,10억뷰 (Feat. MOON),세훈&찬열,"[1,518]",어느 별에서 왔길래 흔들어 놔 자꾸 소름 돋게 하고 맘을 헝클어 놓아 멜로디가 나오...,"['국내', '가요', '봄', '초봄', '달달한', '사랑/기쁨']",0.481553
12580,30702872,"좋아, 고마워 (맥심 화이트골드 - 김연아편 CF 삽입곡)",공기남,"[1,483]",꽃잎이 바람을 타고 흩날리고 있어요 향기로운 꽃냄새가 설레이게 하죠 따사로운 햇살이...,"['국내', '인디', '봄', '국내 인디', '달달한', '봄캐롤', '연애권장...",0.368109
1908,30857459,좋아요 (No Narration),좋아서하는밴드,[11],그대가 좋아요 생각만 해도 좋아요 이러는지 나도 몰라요 그냥 그대와 함께 것만으로 ...,"['국내', '인디', '국내 인디', '화창한 날', '일상', '아침', '주말...",0.362537
15208,30698531,좋아요!,Apink (에이핑크),[696],어제 주고받은 메시지를 자꾸만 보게 돼요 친구랑 있어도 핸드폰만 보게 돼요 왠...,"['국내', '가요', '사랑/기쁨', '썸 탈때', '연애권장송', '달달한', ...",0.334576
1912,31059938,행복한 일은 매일 있어 (With 피터한),타린(Tarin),[135],갓 구운 식빵 따뜻한 커피 오늘따라 좋은 봄바람 작년에 입던 원피스 쏘옥 웬일로 화...,"['국내', '인디', '국내 인디', '화창한 날', '일상', '아침', '주말...",0.272981


In [170]:
music_REC("strawberry moon").head(5)

,song_id,song_name,artist,Like_Count,Lyric,tags,score
12147,30466667,Perfect,pH-1,"[1,496]",뉴욕부터 시차적응 안된 몸 일으키며 나일 탓했어 별 다른말...,"['국내', '가요', '경쾌한', '기분전환', '화창한 날', '신나는', '드...",0.170396
5930,31883146,떠 (Spring),MCND,[107],눈치 볼 뭐가 그냥 가보는 거지 맘대로 직진하다 보면 길이 나와 후진 이젠 믿어...,"['국내', '가요', '아이돌', 'KPOP', '남자아이돌', '청량한', '시...",0.137854
4637,31887893,Cloud 9,CRAVITY,[339],뭔가 다른 느낌 푸른 바다처럼 깊어 색은 짙어질테니 매일 지쳐 뒤척이던 긴긴 밤 이...,"['국내', '가요', '아이돌', 'KPOP', '신나는', '시원한', '드라이...",0.126209
10937,32313718,Shake it (ft.Dislike Mushroom & OLPDio),Hang.9(행구),[1],아래 위로 태워버려 아래 위로 아래 위로 아래 위로 ...,"['국내', '인디', '힙합', '사운드클라우드', '힙스터', '국내 인디', ...",0.104914
8695,5330706,SURFER,1415,"[1,421]",너와 지난날을 기억해 처음엔 알고 싶었어 생각해 너와의 시간은 지났고 얼마전 봤을 ...,"['국내', '인디', '국내 인디', '쌀쌀한 날', '포근한', '초겨울']",0.098627


In [182]:
music_REC("금요일에 만나요 (feat. 장이정 Of History)").head(5)

,song_id,song_name,artist,Like_Count,Lyric,tags,score
6074,30309201,Why So Lonely,원더걸스,"[13,985]",너와 입술이 닿을 때부터 달라질 줄 알았는데 별거 없어 다른 없어 너와 보내는 주말...,"['국내', '걸그룹', 'KPOP', '가요', '여자아이돌', '여름', '시원...",0.141583
1643,9505095,집앞이야 나와,SHY (손호영),[375],하루에 꼭 두통화 적어도 꼭 세통화 하는 말은 어디야 뭐해 알아 뭐하게 알면 오게 ...,"['국내', '가요', '사랑/기쁨', '봄', '달달한', '연애권장송']",0.137674
4654,5937094,My My,세븐틴(SEVENTEEN),"[2,002]",발맞춰 어디론가 건조한 사막이라도 구름은 햇빛의 우산이 하루하루 눈...,"['국내', '가요', '아이돌', 'KPOP', '신나는', '시원한', '드라이...",0.110702
15441,31124876,그대만 보여서,김나영,"[2,898]",눈을 감아도 자꾸 맴도는 미소에 잠을 설치고 그대만 보여 꿈꾸듯 걸음씩 나도 모르게...,"['국내', '달달한', '설렘/심쿵', '사랑/기쁨']",0.103278
12679,4429245,더 가까이,효린,[223],길을 잃은 사람이 건네 준 따스했던 설레임 기억이나 자꾸 생각나 늘 기다린 사랑이라...,"['OST', '영화/드라마', '국내', '빼빼로데이', '드라마', '고백', ...",0.100534


In [167]:
music_REC("미리 메리 크리스마스 (feat. 천둥 Of MBLAQ)").head(5)

,song_id,song_name,artist,Like_Count,Lyric,tags,score
13928,32245079,걸어 (Feat.PEBBLE),WANNA(워나),[2],수많은 걱정들 속에 너라는 이름은 없어 어떻게 그럴 수가 있어 묻는다면 나도 첨이야...,"['국내', '인디', '사랑/기쁨', '국내 인디']",0.174139
826,38793,거꾸로 강을 거슬러 오르는 저 힘찬 연어들처럼,강산에(San-Eh Kang),"[39,881]",흐르는 강물을 거꾸로 거슬러 오르는 연어들의 도무지 그들만의 신비한 이유처럼 언제서...,"['국내', '가요', '복고/레트로', '뉴트로', '감각적인', '신스팝']",0.158437
9652,80394986,엉뚱한 상상,에이트(8Eight),[359],언제부터인지 나도 수는 없지만 성탄절에 눈이 오지 않기 시작했어 매년 그때가 되면 ...,"['국내', '가요', '눈오는 날', '첫눈']",0.127340
11151,32011993,라 루나 (La Luna),핫펠트 (HA:TFELT),[624],막다른 골목에 선 긴장감이 흘러 뻔한 이끌림이래도 운명은 우연의 반복 궁금해져 너...,"['국내', '알앤비', '힙합', '그루브한', '트렌디한', '편집숍/매장', ...",0.123045
9325,5522222,THE LAST HOLIDAY,레이디스 코드(LADIES' CODE),[239],추운 온도에 얼어붙은 손을 내밀어 따뜻한 커피와 추억들을 나눠줄래 떨리는 겨울이지만...,"['국내', '가요', '캐롤', '크리스마스', '눈오는 날', '겨울']",0.119786


## Error
ValueError: The truth value of an array with `more than one element` is `ambiguous`. Use a.any() or a.all()
* 아이유의 좋은 날을 생각하고 입력했던 곡 제목이 출력이 안되는 문제 발견
* 한 개 이상의 요소를 입력하지 않았음에도 `more than one element` is `ambiguous`라는 오류가 발생
* 보유한 노래 목록에서 '좋은 날' 과 일치하는 노래를 출력시켜 보았음

In [186]:
music_REC("좋은 날").head(5)

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [184]:
total_playlist[total_playlist['song_name']=='좋은 날']

,song_id,song_name,artist,album,Like_Count,Lyric,cover_url,tags,words
11846,2150694,좋은 날,아이유(IU),Real,"[40,186]",어쩜 하늘은 파란 건지 오늘따라 바람은 완벽한지 그냥 모르는 척 못들은 척 지워버린...,https://image.bugsm.co.kr/album/images/200/249...,"['국내', 'KPOP', '아이돌', '떼창', '군대', '군인', '밀보드',...","[어쩜, 하늘은, 파란, 건지, 오늘따라, 바람은, 완벽한지, 그냥, 모르는, 척,..."
12972,31177441,좋은 날,멜로망스(MeloMance),미스터 션샤인 (tvN 주말드라마) OST - Part.5,"[4,724]",조용한 바람 생각 물어옵니다 그댄 어느새 맘에 조용히 내려왔죠 깊은 잠일까 꿈을 꾸...,https://image.bugsm.co.kr/album/images/200/201...,"['국내외', '지치고 힘들때', '울고 싶을때', '우울할때', '눈물']","[조용한, 바람, 생각, 물어옵니다, 그댄, 어느새, 맘에, 조용히, 내려왔죠, 깊..."


### 제목이 동일한 곡일 경우 다중으로 입력되어 오류가 발생한 것임을 파악
* 아이유의 '좋은 날' , 멜로망스의 '좋은 날'

---

### 해결 방법 : 곡 제목이 아닌 고유 번호로 검색하는 방식

* 곡 아이디를 str로 형변환

In [444]:
len(total_playlist)

15341

In [445]:
total_playlist.song_id = total_playlist.song_id.astype(str)

In [446]:
len(total_playlist.song_id)

15341

In [447]:
total_playlist.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15341 entries, 0 to 15340
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   song_id     15341 non-null  object
 1   song_name   15341 non-null  object
 2   artist      15341 non-null  object
 3   album       15341 non-null  object
 4   Like_Count  15341 non-null  object
 5   Lyric       15341 non-null  object
 6   cover_url   15341 non-null  object
 7   tags        15341 non-null  object
 8   words       15341 non-null  object
dtypes: object(9)
memory usage: 1.1+ MB


### 인덱스 테이블을 만들 때 곡 이름이 아닌, 고유 번호로 생성

In [518]:
"""TF-IDF로 만들기"""
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer()
tfidf_matrix = tfidf.fit_transform(drop_total_year.Lyric) # DF 변경시 수정
# 줄거리에 대해서 tf-idf 수행
print(tfidf_matrix.shape)

(17891, 161895)


In [519]:
"""코사인 유사도 구하기""" # DF 변경시 수정
from sklearn.metrics.pairwise import linear_kernel
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

* 변경 : index=total_playlist.song_id

In [520]:
"""인덱스 테이블 만들기"""
indices = pd.Series(drop_total_year.index, index=drop_total_year.song_id).drop_duplicates() # DF 변경시 수정
print(indices)

song_id
30392514        0
30061516        1
30506160        2
3988024         3
30428818        4
            ...  
31987127    17886
31977039    17887
32064295    17888
83888993    17889
32381171    17890
Length: 17891, dtype: int64


## 노래 추천 함수 변경 
### 1 곡 명 기준 → 곡 아이디 기준
* 곡 아이디를 입력했을 때 해당 아이디와 일치하는 가사의 코사인 유사도를 계산할 수 있도록 함

### 2 데이터 프레임 구성 변경
* 검색 노래 아이디(rec)를 더하고 추천된 곡의 리스트에서 고유번호, 이름, 아티스트명, 가사, 유사도 점수만 출력되도록 수정
* song_id	/ song_name	 /  artist	/  Lyric	/  score  / rec

In [523]:
"""추천 해주기"""
def music_REC(song_id, cosine_sim=cosine_sim):
    #입력한 노래로부터 인덱스 가져오기
    idx = indices[song_id]

    # 모든 노래에 대해서 해당 노래와의 유사도를 구하기
    sim_scores = list(enumerate(cosine_sim[idx]))

    # 유사도에 따라 노래들을 정렬
    sim_scores = sorted(sim_scores, key=lambda x:x[1], reverse = True)

    # 가장 유사한 10개의 노래를 받아옴
    sim_scores = sim_scores[1:11]

    # 가장 유사한 10개 노래의 인덱스 받아옴
    music_indices = [i[0] for i in sim_scores]
    
    # 기존에 읽어들인 데이터에서 해당 인덱스의 값들을 가져온다. 
    # 그리고 스코어 열을 추가하여 코사인 유사도도 확인할 수 있게 한다.
    result_df = drop_total_year.iloc[music_indices].copy()                 # DF 변경시 수정
    result_df['score'] = [i[1] for i in sim_scores]
    result_df['rec'] = song_id # 추천 리스트 기준 노래 고유 번호 
    
    # 읽어들인 데이터에서 제거할 칼럼명
    # del result_df['content']
    del result_df['cover_url']
    del result_df['album'] 
    del result_df['words'] 
    del result_df['Like_Count'] 
    del result_df['tags'] 
    
    # 가장 유사한 10개의 노래의 제목을 리턴
    return result_df

### 결과 확인
* 아이유 '좋은 날'의 고유 번호는 2150694

In [205]:
music_REC('2150694').head(5)

,song_id,song_name,artist,Like_Count,Lyric,tags,score
8440,80392898,봄봄봄,루싸이트 토끼,"[3,417]",파란하늘 졸린 봄 노란 공기 이런 하기 딱 좋은걸 뻔하고 서툴러 망설여져 지루해도 ...,"['국내', '인디', '국내 인디', '여성보컬', '비온후/맑게갠', '목소리/...",0.147044
2721,3361811,마치 오늘 같으니까,스무살,[191],맞아 그땐 어렸었지 그리고 미쳐 그땐 몰랐었지 그런데 생각해보니까 아니 그럴 필요도...,"['국내', '인디', '잔잔한', '겨울', '카페', '어쿠스틱한', '포크/어...",0.132277
9794,3768051,그녀가 말했다 (With 권진아),토이(Toy),"[6,514]",새 옷을 사고 종일 를 켜놔도 자꾸만 멍하니 전화만 보네요 친굴 만나고 애써 아닌 ...,"['국내', '발라드', '발라드한', '가을', '겨울', '환절기', '감성적인...",0.115737
8876,30249354,우리 이젠 어디로,지소울 (GSoul),"[1,381]",할말 없음 끊을게 졸려 잘게 짜증낸 거 아냐 그냥 피곤해 그만 귀찮게 된 건지 한땐...,"['국내', '인디', '국내 인디', '밤/새벽', '시티팝', '하교/퇴근길',...",0.111458
12758,30724656,어디서부터,이진아,[550],어디서부터 어떻게 할지 자존심은 이리 커져 건지 몰라요 사랑은 어디로 가버린 건지 ...,"['국내', '우울할때', '위로', '나 혼자', '고독', '울고 싶을때', '...",0.099618


* 멜로망스의 '좋은 날'의 고유 번호는 31177441

In [211]:
music_REC('31177441').head(5)

,song_id,song_name,artist,Like_Count,Lyric,tags,score
3540,30047200,걱정말아요 그대,이적,"[16,608]",그대여 아무 걱정 하지 말아요 함께 노래 합시다 아픈 기억들 모두 그대여 가슴에 깊...,"['국내', '발라드한', '발라드', '국내 발라드', '100시리즈', '가요'...",0.224383
6160,31109522,Circus,태연 (TAEYEON),"[4,192]",아주 찬란했고 빛이 가득해 늘 가득해 모두 환호하고 환호해 우릴 향해 모습이...,"['국내', '가요', '아이돌', 'KPOP', '가을', '아련한', '청량한'...",0.224383
9706,30432855,애정 없는 장난,곽푸른하늘,[316],애정 장난에 속아 버렸어 아마 이런 식의 만남은 처음이었을 둘도 존재가 되어주지 않...,"['국내', '발라드', '가을', '이별/슬픔', '발라드한', '국내 발라드']",0.224383
4421,31642988,잠들고 나면,이은서,[86],세계엔 많은 숫자들이 누군가의 뒤엔 누군가의 표정이 나타나고 사라지는 말들이 공기 ...,"['국내', '인디', '국내 인디', '밤/새벽', '카페', '감성적인', '몽...",0.142295
9552,432843,소리,윤상,"[1,097]",좋아하는 몇 가지 있단다 하늘만큼 땅만큼 좋아하는 반가운 사람이 찾아와 문 두드리는...,"['국내', '가요', '발라드한', '국내 발라드', '발라드', '눈오는 날',...",0.136937


In [214]:
music_REC(total_playlist.song_id[0]).head(5)

,song_id,song_name,artist,Like_Count,Lyric,tags,score
1143,30237054,비 오는 거리 (Feat. 핫펠트),베이빌론 (Babylon),"[2,315]",첨 봤을 때부터 이미 시작돼 버렸지 신이 주신 선물이라고 믿고만 싶었었지 한 ...,"['국내', '힙합', '감성힙합', '감성적인', '그루브한', '카페', '비트...",0.180409
2068,5881371,이랬다 저랬다,유주(여자친구),[150],어느 머릿속에 들어와 여기서 사는 자꾸만 눈에 밟히는 맘을 흔들어 놔 뭔가 불편한 ...,"['국내', '가요', '기분전환', '화창한 날', '밝은', '출근/퇴근길', ...",0.150551
11597,2580127,나무가 되는 꿈,박지윤,[465],너와 향한 꿈들이 빛이 되어 달아나 그곳에 어딘가로 떠오를꺼야 우릴 향해 쌓은 노래...,"['국내', '국내 인디', '잔잔한', '밤/새벽', '새벽감성', '잠들기 전']",0.145132
911,31675718,너가 있어,Collective Arts(콜렉티브아츠),[38],너가 품은 웃음이 안겨들고서 서서히 사랑을 속삭이네 아직 서툰 손이 손을 잡고서 서...,"['국내', '인디', '국내 인디', '알앤비', '그루브한', '밤/새벽', '...",0.143039
10785,4470743,"(Bonus Track) 걍 음악이다 Remix (feat. 바스코, 천재노창, B...",C JAMM,[477],음악이다 인마 음악이다 인마 음악이다 인마 음악이다 인마 음악이다 인마 음악이다 인...,"['국내', '힙합', '강한', '그루브한', '스트레스/짜증', '클럽/파티',...",0.128761


---

In [228]:
music_REC('2150694').head(5)

,song_id,song_name,artist,Lyric,score,rec
8931,30722512,"가끔 (Feat. Crush, 기리보이)",긱스(Geeks),요즘은 잠을 가끔은 배가 고파도 밥을 먹을 순 없지 내일 공연이 있으니까 ...,0.147044,2150694
2722,3985935,리얼 러브 송,원 모어 찬스(one more chance),무얼 하려고 해도 많은 시간이 걸려 고민하고 생각해 봤을 생각은 모두 사라지고 하나...,0.132277,2150694
10288,6015665,산막이옛길을 달리는 소녀,위키미키(Weki Meki),함께 매일 똑같은 일상 속 반복되는 지루했던 잠시 멈춰 느낌대로 떠나 ...,0.115737,2150694
9370,82855,Christmas & Christmas,비쥬(Bijou),그대도 보고있나요 하얀눈이 내리고 있어요 살며시 잠든 어린 아이 처럼 세상이 아름다...,0.111458,2150694
13266,5757761,AIR,프롬마스(FROMMARS),매일 아침마다 그리면 공기 속에 불어와 녹이는 목소리 믿을 없어 멀리 떨어져 있...,0.099618,2150694


In [304]:
"""추천 해주기"""
def music_REC(song_id, cosine_sim=cosine_sim):
    #입력한 노래로부터 인덱스 가져오기
    idx = indices[song_id]

    # 모든 노래에 대해서 해당 노래와의 유사도를 구하기
    sim_scores = list(enumerate(cosine_sim[idx]))
    
    # 유사도에 따라 노래들을 정렬
    sim_scores = sorted(sim_scores, key=lambda x:x[1], reverse = True)
    
    # 가장 유사한 10개의 노래를 받아옴
    sim_scores = sim_scores[1:11]

    # 가장 유사한 10개 노래의 인덱스 받아옴
    music_indices = [i[0] for i in sim_scores]
    
    # 기존에 읽어들인 데이터에서 해당 인덱스의 값들을 가져온다. 
    # 그리고 스코어 열을 추가하여 코사인 유사도도 확인할 수 있게 한다.
    result_df = total_playlist.iloc[music_indices].copy()
    result_df['score'] = [i[1] for i in sim_scores]
    result_df['rec'] = song_id # 추천 리스트 기준 노래 고유 번호 
    
    # 읽어들인 데이터에서 제거할 칼럼명
    # del result_df['content']
    del result_df['cover_url']
    del result_df['album'] 
    del result_df['words'] 
    del result_df['Like_Count'] 
    del result_df['tags'] 

    # 가장 유사한 10개의 노래의 제목을 리턴
    return result_df

---

## 다른 함수 적용 (활용 X)

In [424]:
row_number = pd.Series(total_playlist.index, index = total_playlist['song_id']).drop_duplicates()
row_number.head(10)

song_id
30392514    0
30061516    1
30506160    2
3988024     3
30428818    4
2587551     5
4311833     6
30329063    7
30548970    8
30355243    9
dtype: int64

In [410]:
len(total_playlist)

15341

In [432]:
len(row_number)

15341

In [433]:
row_number.max()

15340

In [430]:
"""추천 해주기"""
def music_REC(song_id, cosine_sim=cosine_sim):
    #입력한 노래로부터 인덱스 가져오기
    #idx = indices[song_id]
    idx = row_number[song_id]
    # 모든 노래에 대해서 해당 노래와의 유사도를 구하기
    #sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = list(enumerate( cosine_sim[idx]))
    # 유사도에 따라 노래들을 정렬
    # sim_scores = sorted(sim_scores, key=lambda x:x[1], reverse = True)
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    
    # 가장 유사한 10개의 노래를 받아옴
    # sim_scores = sim_scores[1:11]
    sim_scores = sim_scores[0:10]
    print(sim_scores)
    # 가장 유사한 10개 노래의 인덱스 받아옴
    #music_indices = [i[0] for i in sim_scores]
    music_idx = [n[0] for n in sim_scores]
    # 기존에 읽어들인 데이터에서 해당 인덱스의 값들을 가져온다. 
    # 그리고 스코어 열을 추가하여 코사인 유사도도 확인할 수 있게 한다.
    result_df = total_playlist.iloc[music_idx].copy()
    
    # result_df = total_playlist.iloc[music_indices].copy()
    result_df['score'] = [i[1] for i in sim_scores]
    result_df['rec'] = song_id # 추천 리스트 기준 노래 고유 번호 
    
    # 읽어들인 데이터에서 제거할 칼럼명
    # del result_df['content']
    del result_df['cover_url']
    del result_df['album'] 
    del result_df['words'] 
    del result_df['Like_Count'] 
    del result_df['tags'] 

    # 가장 유사한 10개의 노래의 제목을 리턴
    return result_df

---

## Error
IndexError: positional indexers are out-of-bounds

In [229]:
test = total_playlist[:5]

In [383]:
result =[]
for i in tqdm(range(len(total_playlist))):
    result.append(music_REC(total_playlist.song_id.iloc[i]))

  0%|          | 28/15341 [00:00<02:26, 104.26it/s]


IndexError: positional indexers are out-of-bounds

---

In [361]:
len(total_playlist)

15341

In [456]:
result =[]
for i in tqdm(range(len(test))):
    result.append(music_REC(test.song_id.iloc[i]))

100%|██████████| 5/5 [00:00<00:00, 111.60it/s]


In [459]:
pd.concat(result)

,song_id,song_name,artist,Lyric,score,rec
1143,30237054,비 오는 거리 (Feat. 핫펠트),베이빌론 (Babylon),첨 봤을 때부터 이미 시작돼 버렸지 신이 주신 선물이라고 믿고만 싶었었지 한 ...,0.180096,30392514
2068,5881371,이랬다 저랬다,유주(여자친구),어느 머릿속에 들어와 여기서 사는 자꾸만 눈에 밟히는 맘을 흔들어 놔 뭔가 불편한 ...,0.150344,30392514
10606,32114315,나쁜년,안재우,혼자 두지 말아줘요 하룰 넘어 매일이 비슷해요 찬 바람이 불던 때는 그때였을 둘이 ...,0.144952,30392514
911,31675718,너가 있어,Collective Arts(콜렉티브아츠),너가 품은 웃음이 안겨들고서 서서히 사랑을 속삭이네 아직 서툰 손이 손을 잡고서 서...,0.141827,30392514
9796,31613111,FUN!,프로미스나인,맘이 사르르르 땀이 주르르륵 머린 핑그르르 이래 이래 펑 터질 것만 펑 쏘아 올...,0.128566,30392514
416,30186752,내가 이래,한해,오랜만이네 오 년 만인거 같애 그럭저럭 지내 어떻게 지냈어 달라 보인다 시간은 잔인...,0.127743,30392514
12814,31323300,괜히,민수,괜히 한번 걸어봤어 새벽에 흔드려는 아니고 맘을 주체할 수가 없어서 차분했다가 벅차...,0.125463,30392514
5988,32346842,필요해 (solo ver.),OoOo(오넷),그냥 너가 필요해 밤들이 외롭게 그냥 너가 있음 늘 옆에 너가 날들이 괴롭게 ...,0.121220,30392514
7197,2689027,보고싶어,목요일 아침 밴드(Thursday Morning Band),보고싶어 매일 생각해 잊은건지 눈물이 흘러 만난 표현할 사랑이 내속에 와서 부둥켜 ...,0.118000,30392514
1966,6030564,Oh I Know,뉘뉘(NUITNUIT),가끔 모습은 외로워보이고 보고 무슨 말을 하려는지 말은 하지마 맘이 잠기고있잖아 아...,0.113368,30392514


In [457]:
pd.concat(result).groupby('rec')['song_name'].count()

rec
30061516    10
30392514    10
30428818    10
30506160    10
3988024     10
Name: song_name, dtype: int64

In [467]:
sort_total_years = pd.read_csv('sort_total_years.csv')

In [474]:
words = []
for word in sort_total_years['Lyric'].tolist():
    words.append(word.split())
    words = [each_word for each_word in words if each_word not in stop_w]
#create the new column with the information of words lists
sort_total_years['words'] = words
sort_total_years.head()

,song_id,song_name,artist,album,Like_Count,Lyric,cover_url,tags,year,words
0,191162,오빠는 풍각(風角)쟁이,박향림,유성기로 듣던 불멸의 명가수 - 박향림 편,1914,오빠는 풍각쟁이야 머 오빠는 심술쟁이야 머 난몰라 난몰라 내반찬 다 뺏어 먹는거...,https://image.bugsm.co.kr/album/images/200/101...,"['연도별 인기가요', '올디스', '연대별', '연대별 음악']",1940y,"[오빠는, 풍각쟁이야, 머, 오빠는, 심술쟁이야, 머, 난몰라, 난몰라, 내반찬, ..."
1,80078168,애수의 소야곡,남인수,남인수 골든,937,운다고 옛사랑이 오리요만은 눈물로 달래보는 구슬픈 이밤 고요히 창을 열고 별빛...,https://image.bugsm.co.kr/album/images/200/800...,"['연도별 인기가요', '올디스', '연대별', '연대별 음악']",1940y,"[운다고, 옛사랑이, 오리요만은, 눈물로, 달래보는, 구슬픈, 이밤, 고요히, 창을..."
2,111683,불효자는 웁니다,진방남,"유성기로 듣던 불멸의 명가수 - 진방남, 고운봉, 송달협 편",732,울며헤진 부산항을 돌아다보니 연락선 난간머리 흘러온달빛 이별만은 어렵더라 이별만...,https://image.bugsm.co.kr/album/images/200/101...,"['연도별 인기가요', '올디스', '연대별', '연대별 음악']",1940y,"[울며헤진, 부산항을, 돌아다보니, 연락선, 난간머리, 흘러온달빛, 이별만은, 어렵..."
3,173791,눈물젖은 두만강(豆滿江),김정구,유성기로 듣던 불멸의 명가수 - 김정구 편,609,두만강 푸른 물에 노젓는 뱃사공 흘러간 그 옛날에 내 님을 싣고 ...,https://image.bugsm.co.kr/album/images/200/101...,"['연도별 인기가요', '올디스', '연대별', '연대별 음악']",1940y,"[두만강, 푸른, 물에, 노젓는, 뱃사공, 흘러간, 그, 옛날에, 내, 님을, 싣고..."
4,173084,번지없는 주막,백년설,유성기로 듣던 불멸의 명가수 - 백년설 편,511,불러봐도 울어봐도 못 오실 어머님을 원통해 불러보고 땅을 치며 통곡해요 다시 ...,https://image.bugsm.co.kr/album/images/200/100...,"['연도별 인기가요', '올디스', '연대별', '연대별 음악']",1940y,"[불러봐도, 울어봐도, 못, 오실, 어머님을, 원통해, 불러보고, 땅을, 치며, 통..."


---

## join

## 형변환 astype
* song_id : int → str
* Like_Count : str → int

## total_playlist

In [477]:
total_playlist.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15341 entries, 0 to 15340
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   song_id     15341 non-null  object
 1   song_name   15341 non-null  object
 2   artist      15341 non-null  object
 3   album       15341 non-null  object
 4   Like_Count  15341 non-null  object
 5   Lyric       15341 non-null  object
 6   cover_url   15341 non-null  object
 7   tags        15341 non-null  object
 8   words       15341 non-null  object
dtypes: object(9)
memory usage: 1.1+ MB


In [485]:
total_playlist['Like_Count'] = total_playlist['Like_Count'].str.replace('\[', "").replace('\]', '', regex= True).replace(',', '', regex= True)
total_playlist['Like_Count'] = total_playlist['Like_Count'].astype(int)

In [487]:
total_playlist.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15341 entries, 0 to 15340
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   song_id     15341 non-null  object
 1   song_name   15341 non-null  object
 2   artist      15341 non-null  object
 3   album       15341 non-null  object
 4   Like_Count  15341 non-null  int64 
 5   Lyric       15341 non-null  object
 6   cover_url   15341 non-null  object
 7   tags        15341 non-null  object
 8   words       15341 non-null  object
dtypes: int64(1), object(8)
memory usage: 1.1+ MB


In [490]:
total_playlist['year'] = np.nan

In [492]:
total_playlist.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15341 entries, 0 to 15340
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   song_id     15341 non-null  object 
 1   song_name   15341 non-null  object 
 2   artist      15341 non-null  object 
 3   album       15341 non-null  object 
 4   Like_Count  15341 non-null  int64  
 5   Lyric       15341 non-null  object 
 6   cover_url   15341 non-null  object 
 7   tags        15341 non-null  object 
 8   words       15341 non-null  object 
 9   year        0 non-null      float64
dtypes: float64(1), int64(1), object(8)
memory usage: 1.2+ MB


In [494]:
total_playlist.head(1)

,song_id,song_name,artist,album,Like_Count,Lyric,cover_url,tags,words,year
0,30392514,bath,offonoff (오프온오프),bath,5781,거긴 날씨가 어때 하늘과 밤엔 자꾸 떠오르는 기억이 계속 말을 건네네 우리는 함께일...,https://image.bugsm.co.kr/album/images/200/200...,"['국내', '힙합', '알앤비', '밤/새벽', '감각적인', '작업실', '몽환...","[거긴, 날씨가, 어때, 하늘과, 밤엔, 자꾸, 떠오르는, 기억이, 계속, 말을, ...",NaN


In [499]:
total_playlist.to_csv('total_playlist.csv',index=False)

---

## sort_total_years

In [480]:
sort_total_years.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4562 entries, 0 to 4561
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   song_id     4562 non-null   int64 
 1   song_name   4562 non-null   object
 2   artist      4562 non-null   object
 3   album       4562 non-null   object
 4   Like_Count  4562 non-null   int64 
 5   Lyric       4562 non-null   object
 6   cover_url   4562 non-null   object
 7   tags        4562 non-null   object
 8   year        4562 non-null   object
 9   words       4562 non-null   object
dtypes: int64(2), object(8)
memory usage: 356.5+ KB


In [481]:
sort_total_years.Like_Count = sort_total_years.Like_Count.astype(int)
sort_total_years.song_id = sort_total_years.song_id.astype(str)

In [482]:
sort_total_years.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4562 entries, 0 to 4561
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   song_id     4562 non-null   object
 1   song_name   4562 non-null   object
 2   artist      4562 non-null   object
 3   album       4562 non-null   object
 4   Like_Count  4562 non-null   int64 
 5   Lyric       4562 non-null   object
 6   cover_url   4562 non-null   object
 7   tags        4562 non-null   object
 8   year        4562 non-null   object
 9   words       4562 non-null   object
dtypes: int64(1), object(9)
memory usage: 356.5+ KB


In [498]:
sort_total_years.to_csv('sort_total_years.csv',index=False)

---

In [495]:
total_year = total_playlist.append(sort_total_years).reset_index(drop=True)

In [497]:
display(total_year.head(2),total_year.tail(2))

,song_id,song_name,artist,album,Like_Count,Lyric,cover_url,tags,words,year
0,30392514,bath,offonoff (오프온오프),bath,5781,거긴 날씨가 어때 하늘과 밤엔 자꾸 떠오르는 기억이 계속 말을 건네네 우리는 함께일...,https://image.bugsm.co.kr/album/images/200/200...,"['국내', '힙합', '알앤비', '밤/새벽', '감각적인', '작업실', '몽환...","[거긴, 날씨가, 어때, 하늘과, 밤엔, 자꾸, 떠오르는, 기억이, 계속, 말을, ...",NaN
1,30061516,[권리없는 곡],아파치(Apachi),Toujours Youth Remix Album,54,겁이 아무 말이나 들려줘 순전히 아름다움 팍 튀고 팍 어쩔 없이 감...,https://image.bugsm.co.kr/album/images/200/200...,"['국내', '힙합', '알앤비', '밤/새벽', '감각적인', '작업실', '몽환...","[겁이, 아무, 말이나, 들려줘, girl, i'm, tired, lgbt, nex...",NaN


,song_id,song_name,artist,album,Like_Count,Lyric,cover_url,tags,words,year
19901,83888993,Birthday,Festy Wxs,Birthday,0,어젯밤 그녀는 빛을 보고 빨개 날개 달린 건 모조리 다 내 곁에 말해 뭘 원해 ...,https://image.bugsm.co.kr/album/images/200/163...,"['NewSongs', ""IT'S NEW! 신곡 에센셜""]","[어젯밤, 그녀는, 빛을, 보고, 빨개, 날개, 달린, 건, 모조리, 다, 내, 곁...",2021y
19902,32381171,Noise (feat. 이츠),Opius,purple rain,0,네가 원하는 걸 내게 말해줘 네 손을 잡고 달려줄 수 있어 ...,https://image.bugsm.co.kr/album/images/200/204...,"['NewSongs', ""IT'S NEW! 신곡 에센셜""]","[네가, 원하는, 걸, 내게, 말해줘, 네, 손을, 잡고, 달려줄, 수, 있어, 혼...",2021y


In [500]:
total_year.drop_duplicates(['song_id','song_name'], keep='first',inplace=True)

In [501]:
len(total_year)

18090

In [507]:
total_year[total_year.song_name=='Something']

,song_id,song_name,artist,album,Like_Count,Lyric,cover_url,tags,words,year
27,3336350,Something,걸스데이(Girl's Day),걸스데이 미니앨범 3집,2595,흔들리는 표정 말투에 뭔가 듯해 몸에 베인 낯선 향기에 뭔가에 홀린 듯해 촉에 찔리...,https://image.bugsm.co.kr/album/images/200/405...,"['국내', '가요', '아이돌', 'KPOP', '섹시한']","[don't, look, into, eyes, lie, again, i'm, sic...",NaN
227,4447671,Something,걸스데이(Girl's Day),Girl's Day Love Second Album,490,흔들리는 표정 말투에 뭔가 듯해 몸에 베인 낯선 향기에 뭔가에 홀린 듯해 촉에 찔리...,https://image.bugsm.co.kr/album/images/200/515...,"['국내', '가요', '섹시한']","[don't, look, into, eyes, lie, again, i'm, sic...",NaN
14860,5342023,Something,죠지,내 아이디는 강남미인 (JTBC 금토드라마) OST - Part.4,918,시간이 멈춰버린 느낌 요즘 따라 그댈 마주할때 늘 마음이 흘러가는대로 너와 서로 스...,https://image.bugsm.co.kr/album/images/200/779...,"['국내', '인디', '설렘/심쿵', '감성적인', '사랑/기쁨', '국내 인디']","[something, your, eyes, tell, who, i, and, som...",NaN
17664,3860,Something,써클(Circle),Magic Eyes,762,천천히 내게 다가와 그런 사랑은 정말 싫어 내모습을 봐 살며시 조급히 내게로 오...,https://image.bugsm.co.kr/album/images/200/1/1...,"['국내', '댄스', '아이돌', '연대별', '신나는', '100시리즈', '여...","[천천히, 내게, 다가와, 그런, 사랑은, 정말, 싫어, 내모습을, 봐, 살며시, ...",2000y
18864,31947114,Something,강다니엘(KANG DANIEL),편의점 샛별이 OST Part.1,2378,깨지 못하게 날 가둬 난 빠져 숱한 널 향한 감정들이 날 흔들어 아무도 모르게 ...,https://image.bugsm.co.kr/album/images/200/203...,"['2020년대', '2020년', '영화/드라마', '드라마', '연도별 국내드라...","[깨지, 못하게, 날, 가둬, 난, 빠져, 숱한, 널, 향한, 감정들이, 날, 흔들...",2020y


In [509]:
drop_total_year = total_year.drop_duplicates(['Lyric'], keep='first',inplace=False)

In [511]:
total_year.Lyric.duplicated().sum()

199

In [512]:
drop_total_year.Lyric.duplicated().sum()

0

In [514]:
drop_total_year.reset_index(drop=True,inplace=True)

In [516]:
len(drop_total_year)

17891

In [515]:
from tqdm import tqdm

In [524]:
result =[]
for i in tqdm(range(len(drop_total_year))):
    result.append(music_REC(drop_total_year.song_id[i]))
    

100%|██████████| 17891/17891 [02:56<00:00, 101.57it/s]


In [525]:
rec_music = pd.concat(result)

In [526]:
rec_music.to_csv('rec_10_total_years.csv',index=False)

In [527]:
rec_music

,song_id,song_name,artist,Lyric,year,score,rec
1133,30237054,비 오는 거리 (Feat. 핫펠트),베이빌론 (Babylon),첨 봤을 때부터 이미 시작돼 버렸지 신이 주신 선물이라고 믿고만 싶었었지 한 ...,NaN,0.182691,30392514
2057,5881371,이랬다 저랬다,유주(여자친구),어느 머릿속에 들어와 여기서 사는 자꾸만 눈에 밟히는 맘을 흔들어 놔 뭔가 불편한 ...,NaN,0.153537,30392514
10513,32114315,나쁜년,안재우,혼자 두지 말아줘요 하룰 넘어 매일이 비슷해요 찬 바람이 불던 때는 그때였을 둘이 ...,NaN,0.147076,30392514
902,31675718,너가 있어,Collective Arts(콜렉티브아츠),너가 품은 웃음이 안겨들고서 서서히 사랑을 속삭이네 아직 서툰 손이 손을 잡고서 서...,NaN,0.142879,30392514
9716,31613111,FUN!,프로미스나인,맘이 사르르르 땀이 주르르륵 머린 핑그르르 이래 이래 펑 터질 것만 펑 쏘아 올...,NaN,0.130711,30392514
...,...,...,...,...,...,...,...
6535,31926716,7년 동안,뭉크 (MUNKH),생각나 처음 만난 그날이 웃음 가득 채운 모습을 그땐 몰랐지 여자 될 거 란 벌써 ...,NaN,0.143392,32381171
17223,31802125,너를 사랑하고 있어,백현(BAEKHYUN),어떤 말로 표현해야 이 감정을 알까 어떻게 널 바라봐야 나의 마음을 알까 너와 ...,2020y,0.138545,32381171
13461,30075574,혼자 두지 마,백예린 (Yerin Baek),아침에 일어나 혼자인 기분은 말로 표현할 없을 만큼 외롭고 왠지 슬퍼 왠지 모르게 ...,NaN,0.136379,32381171
17030,3547099,한 번쯤 니가 먼저,바닐라 어쿠스틱(Vanilla Acoustic),오늘도 전화기를 보다 내가 먼저 문자 답이 없는 넌 참 혹시 네 맘엔 내가 없...,2010y,0.131689,32381171


---